In [2]:
#/usr/bin/env python

import sqlite3
from sqlite3 import Error
import datetime
from datetime import timedelta
import pandas as pd
from pandas import Series, DataFrame
import pandas_datareader.data as web



In [7]:
def get_symbol_data_today(now,sym):
    
    #start=%s/%s/%s" % (i.day, i.month, i.year)
    prev = datetime.datetime.now() - timedelta(days=3)
    start=datetime.datetime(prev.year,prev.month,prev.day)
    end=datetime.datetime(now.year,now.month,now.day)
    try:
        f = web.DataReader(sym, 'yahoo', start,end)
    except Error as e:
        print(e)    
    
    #f=f.set_index(['Date'])
    #r=web.get_quote_yahoo('AAPL')
    return f.tail(1)

def update_data_to_db(conn,sym):
    sym="NVDA"
    now = datetime.datetime.now()
    data = get_symbol_data_today(now,sym)
    #print(data.keys())
    t=data['Open'].values[0]
    t1=data['High'].values[0]
    t2=data['Low'].values[0]
    t3=data['Close'].values[0]
    t4=data['Adj Close'].values[0]
    t5=data['Volume'].values[0]
    print(t,t1,t2,t3,t4,t5)
    #'Open'
    print("Open : ", data['Open'])
    #'High'
    print("High : ", data['High'])
    #'Low'
    print("Low : ", data['Low'])
    #'Close'
    print("Close : ", data['Close'])
    #'Adj Close'
    print("Adj Close : ", data['Adj Close'])
    #'Volume
    print("Volume : ", data['Volume'])
    
    symbolid = get_symbol_id(conn, sym)
    
    stockpricesinfo = (now,symbolid,t,t1,t2,t3,t4,t5)
    id = insert_stockprices(conn, stockpricesinfo)
    print_stockprices(conn)
    
def get_symbol_id(conn,sym):
    
    cur = conn.cursor()
    cur.execute("SELECT id FROM stockdetails WHERE companysymbol=?",(sym,))
    row=cur.fetchone()
    print(row[0])    
    return row[0]     
    
def insert_stockprices(conn,stockpricesinfo):    
    """
    Create a new stockprices into the stockprices table
    :param conn:
    :param stockprices:
    :return: id
    """
    sql = ''' INSERT INTO stockprices(date,stockdetails_id,openvalue,highvalue,lowvalue,closevalue,adjclose,volume)
              VALUES(?,?,?,?,?,?,?,?) ''' 
    
    cur = conn.cursor()
    cur.execute(sql, stockpricesinfo)
    
    return cur.lastrowid 

def print_stockprices(conn):
    """
    Query all rows in the stockprices table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM stockprices")
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    
        
def create_connection(db):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db)
        print("Database opened successfully")
        return conn
    except Error as e:
        print(e)
 
    return None

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def close_connection(db_conn):
    """ Close connection to the database """
    db_conn.close()   
    
def insert_stockdetails(conn, stockinfo):
    """
    Create a new stockdetails into the stockdetails table
    :param conn:
    :param stockdetails:
    :return: id
    """
    sql = ''' INSERT INTO stockdetails(companysymbol,companyname)
              VALUES(?,?) '''
    
    cur = conn.cursor()
    cur.execute(sql, stockinfo)
    
    return cur.lastrowid 

                
def print_stockdetails(conn):
    """
    Query all rows in the stockdetails table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM stockdetails")
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
                
      
               
def main():
    
       
    database = "/Users/Ravi/Downloads/stock_backend/stocksqlite.db"
 
    stockdetails_table = """CREATE TABLE IF NOT EXISTS stockdetails (
                                        id integer PRIMARY KEY,
                                        companysymbol varchar(10) NOT NULL,
                                        companyname text NOT NULL
                                        
                                    );"""                            
                                    
 
    stockprices_table = """CREATE TABLE IF NOT EXISTS stockprices (
                                    id integer PRIMARY KEY,
                                    date date NOT NULL,
                                    stockdetails_id interger NOT NULL,
                                    openvalue real NOT NULL,
                                    highvalue real NOT NULL,
                                    lowvalue real NOT NULL,
                                    closevalue real NOT NULL,
                                    adjclose real NOT NULL,
                                    volume integer NOT NULL,
                                    FOREIGN KEY (stockdetails_id) REFERENCES stockdetails (id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create stockdetails table
        create_table(conn, stockdetails_table)
        # create stockprices table
        create_table(conn, stockprices_table)
    else:
        print("Error! cannot create the database connection.")

    with conn:
        # create a new stockdetails
        #stockinfo = ('NVDA', 'Nvidia Corporataion');
        #id = insert_stockdetails(conn, stockinfo)
        print_stockdetails(conn)
        #while loop - get symbols from st
        # create a new stockprices
        sym = "NVDA"
        stockprices = update_data_to_db(conn, sym)
    # close database connection      
    close_connection(conn)

if __name__ == '__main__':
    main()


Database opened successfully
(1, 'NVDA', 'Nvidia Corporataion')
225.0 226.270004 223.210007 224.080002 224.080002 10138568
Open :  Date
2018-01-11    225.0
Name: Open, dtype: float64
High :  Date
2018-01-11    226.270004
Name: High, dtype: float64
Low :  Date
2018-01-11    223.210007
Name: Low, dtype: float64
Close :  Date
2018-01-11    224.080002
Name: Close, dtype: float64
Adj Close :  Date
2018-01-11    224.080002
Name: Adj Close, dtype: float64
Volume :  Date
2018-01-11    10138568
Name: Volume, dtype: int64
1
(1, '2018-01-11 16:18:02.310223', 1, 225.0, 226.270004, 223.210007, 224.080002, 224.080002, b'\xc8\xb3\x9a\x00\x00\x00\x00\x00')
